In [ ]:
from lib import *

from xgboost.callback import TrainingCallback
from xgboost.callback import LearningRateScheduler


class myTrainingMonitor(xgb.callback.TrainingCallback):
    def __init__(self, xgbmodel, test_used = False):
        self.xgbmodel = xgbmodel
        self.test_used = test_used
        pass
    
    def after_iteration(self, model, epoch, evals_log):
        if self.test_used:
            print(epoch, 'auc train:', evals_log['eval_train']['auc'][-1], 
                  'auc test:', evals_log['eval_test']['auc'][-1])
        else:
            print(epoch, 'auc train:', evals_log['eval_train']['auc'][-1])            



class myLRScheduler(xgb.callback.TrainingCallback):
    def __init__(self, xgbmodel, learning_rate):
        #super().__init__(lr_scheduler)
        self.xgbmodel = xgbmodel

        self.last_update = 0
        self.minimum_improvement = 0.00000010
        self.lr = learning_rate
        
        # for thr_improvement func
        self.min_impr_for_lr = 0.0001
        
        self.best_model = None
        self.best_performance = 0
        self.best_epoch = -1
        self.n_epochs = 0
             
    
    def giveLr(self, epoch):        
        #print('lr:',self.lr)
        return self.lr
    
    
    def after_iteration(self, model, epoch, evals_log):
        if evals_log['eval_test']['auc'][-1] > self.best_performance:
            self.best_performance = evals_log['eval_test']['auc'][-1]
            self.best_performance_train = evals_log['eval_train']['auc'][-1]
            self.best_model = model.copy()
            self.best_epoch = epoch
            
        #self.thr_improvement(model, epoch, evals_log)
        return self.no_impr_rounds(model, epoch, evals_log)
            
    
    def no_impr_rounds(self, model, epoch, evals_log):
        self.n_epochs = epoch
        
        if (epoch - self.last_update) > 5:
            no_impr = True
             
            for i in range(-5,0):
                if  evals_log['eval_test']['auc'][-6] < evals_log['eval_test']['auc'][i]:
                    no_impr = False
                    break
            
            if no_impr:
                if self.lr < 0.0001:
                    return True
                
                self.lr /= 1.3
                self.last_update = epoch
                #print('no impr')
                #print('new lr:', self.lr)
                
    
    def thr_improvement(self, model, epoch, evals_log):        
        if (epoch - self.last_update) > 5:  
            minus_one = evals_log['eval_test']['auc'][-1]
            minus_five = evals_log['eval_test']['auc'][-5]
            diff = evals_log['eval_test']['auc'][-1] - evals_log['eval_test']['auc'][-5]
            print('diff:',diff, minus_one - minus_five)
            print('-1:',evals_log['eval_test']['auc'][-1])
            print('-5:',evals_log['eval_test']['auc'][-5])
            
            if diff < self.minimum_improvement and self.lr <= 0.00001:
                return True

            if diff < self.min_impr_for_lr:
                self.last_update = epoch
                self.lr /= 10
                self.min_impr_for_lr /= 10 
                
                print('----NEW LEARNING RATE APPLIED---------')
                print(self.lr)
                
                
    def after_training(self, model):
        #print('best auc:', self.best_performance)
        #print('best epoch:', self.best_epoch)
        
        return self.best_model
        
        
        
class XGBWrapper:
    def __init__(self):
        pass
    
    def train(self, train_m, n_estimators, max_depth, learning_rate, eval_m = None, 
              booster = 'gbtree',  scale_pos_weight = 1.0, lambdaa = 1.0, alpha = 0, gamma = 0,
              min_child_weight = 1, colsample_bytree = 1.0):
        
        params = {}
        params['booster'] = booster
        params['scale_pos_weight'] = scale_pos_weight
        params['lambda'] = lambdaa
        params['alpha'] = alpha
        params['gamma'] = gamma
        params['objective'] = 'binary:logistic'
        params['eval_metric'] = 'auc'
        params['booster'] = 'gbtree'
        params['max_depth'] = max_depth
        params['min_child_weight'] = min_child_weight
        params['colsample_bytree'] = colsample_bytree
        params['seed'] = 10
        #params['eta'] = learning_rate
        #params['tree_method'] = 'exact'
        
        if eval_m == None:
            evallist = [(train_m, 'eval_train')]
            trainingCallback = myTrainingMonitor(self, False)
        else:
            evallist = [(train_m, 'eval_train'),(eval_m, 'eval_test')]
            trainingCallback = myTrainingMonitor(self, True)
            
        lrClass = myLRScheduler(self, learning_rate)
        lrCallback = xgb.callback.LearningRateScheduler(lrClass.giveLr)
        
        num_round = n_estimators        
        
        start = time.time()
        model = xgb.train(params,train_m,num_round,  evals = evallist, 
                          callbacks=[lrClass, lrCallback], 
                          early_stopping_rounds = 500, verbose_eval = False)
        
        model = lrClass.best_model
        
        self.best_performance = lrClass.best_performance
        self.best_performance_train = lrClass.best_performance_train
        self.best_epoch = lrClass.best_epoch
        self.n_epochs = lrClass.n_epochs
                
        
        self.model = model
        end = time.time()
        self.elapsed_time = end - start
        
    def predict(self, X_test):
        p = xgb.DMatrix(X_test)
        return self.model.predict(p)

    
    
    
t = CEventsTable()
t.appendFromCsv("ivan_ntuples/tH.csv",1,"tH")
t.appendFromCsv("ivan_ntuples/ttb.csv",0,"ttb")
t.appendFromCsv("ivan_ntuples/ttc.csv",0,"ttc")
t.appendFromCsv("ivan_ntuples/ttL.csv",0,"ttL")
t.appendFromCsv("nazim_ntuples/ttH.csv",0,"ttH")
t.appendFromCsv("nazim_ntuples/ttZ.csv",0,"ttZ")
t.appendFromCsv("ivan_ntuples/ttW.csv",0,"ttW")
t.appendFromCsv("ivan_ntuples/tZq.csv",0,"tZq")
t.appendFromCsv("ivan_ntuples/tWZ.csv",0,"tWZ")
t.appendFromCsv("ivan_ntuples/tW.csv",0,"single_tW")
t.appendFromCsv("ivan_ntuples/single_t1.csv",0,"single_tt")
t.appendFromCsv("ivan_ntuples/single_t2.csv",0,"single_tt")
t.appendFromCsv("ivan_ntuples/single_ts.csv",0,"single_ts")
t.appendFromCsv("nazim_ntuples/WZ.csv",0,"WZ")
t.appendFromCsv("nazim_ntuples/VV1.csv",0,"VV")
t.appendFromCsv("nazim_ntuples/VV2.csv",0,"VV")
t.appendFromCsv("ivan_ntuples/data.csv",0,"nonp")

t.subSample('ttH', 0.1)



print('Before preselection:')
t.printTypesNumbers()
t.applyPreselection()
print('After preselection:')
t.printTypesNumbers()
print('With weights:')
t.calculateWeights()
t.printTypesNumbers()    
    
    
mld = CSampledDatasetExpEvents(t.table, (20000,100000), (10000,100000))
#mld = CSampledDatasetAcc(t.table, (20000,100000), (10000,100000))
mld.printInfo()


X_train = mld.X_train
y_train = mld.y_train
types_train = mld.types_train
weights_train = mld.weights_train

X_test = mld.X_test
y_test = mld.y_test
types_test = mld.types_test
weights_test = mld.weights_test


X_val = mld.X_val
y_val = mld.y_val
types_val = mld.types_val
weights_val = mld.weights_val


dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
dval = xgb.DMatrix(X_val, label=y_val)



AucCalc = CEvaluationAUC()
SigCalc = CEvaluationSignificance()

In [ ]:
class CObjective:
    def __init__(self, filepath):
        self.filepath = filepath
        self.df = pd.DataFrame()
        self.df.to_csv(self.filepath, index=False)

    def __call__(self, trial):
            
        # booster, def gbtree
        #booster = trial.suggest_categorical('booster', ['gbtree', 'gblinear', 'dart'])
        booster = 'gbtree'

        # maximalni hloubka, def 6
        #max_depth = trial.suggest_int('max_depth', 3, 7)
        max_depth = 5

        # vaha kladne tridy, def 1
        #scale_pos_weight =  trial.suggest_float('scale_pos_weight', 1.0, 5.0)
        scale_pos_weight = 1.0

        # L2 regularizace pro vahy, def 1
        lambdaa =  trial.suggest_float('lambda', 0, 15.0)

        # L1 regularizace pro vahy, def 0
        # poznamka - rozsiren rozsah, protoze predchozi hodnoty byly blizko 10
        alpha =  trial.suggest_float('alpha', 0, 50.0)

        # Minimum loss reduction required to make a further partition on a leaf node of the tree, def 0
        gamma =  trial.suggest_float('gamma', 0, 8.0)


        # min_child_weight - Minimum sum of instance weight (hessian) needed in a child.
        # If the tree partition step results in a leaf node with the sum of instance weight less than min_child_weight, 
        # then the building process will give up further partitioning. def 1
        min_child_weight = trial.suggest_float('min_child_weight', 0, 20.0)

        # colsample_bytree is the subsample ratio of columns when constructing each tree. def 1
        colsample_bytree = 0.6 #trial.suggest_float('colsample_bytree', 0.5, 1.0)


        model = XGBWrapper()
        model.train(dtrain, 10000, max_depth, 0.1, dval, 
                    scale_pos_weight = scale_pos_weight, lambdaa = lambdaa,
                    alpha = alpha, gamma = gamma, booster = booster, 
                    min_child_weight = min_child_weight, colsample_bytree = colsample_bytree)

        best_performance = model.best_performance
        best_performance_train = model.best_performance_train
        best_epoch = model.best_epoch
        n_epochs = model.n_epochs
        el_time =  model.elapsed_time

        print('\ntraining:')
        print('best auc:',best_performance,'best epoch:', best_epoch, 'epoch cnt:', n_epochs, 
              'time:',el_time, 'best train auc:', best_performance_train)

        preds = model.predict(X_test)

        auc = AucCalc.evaluate(preds, y_test)

        significance, thr, sig, bg = SigCalc.evaluate(preds, y_test, weights_test)

        print('test auc:', auc)
        print('test sig:', significance)

        line = pd.DataFrame({  'min_child_weight' : [min_child_weight], 'colsample_bytree' : [colsample_bytree],
                  'lambda': [lambdaa], 'alpha':[alpha], 'gamma' : [gamma], 'best_auc_val':[best_performance], 'best_auc_train':[best_performance_train],
                  'auc_test':[auc], 'best_significance':[significance], 'n_sig':[sig],'n_bg':[bg],'thr':[thr], 
                  'n_epochs':[n_epochs], 'best_epoch': [best_epoch], 'time':[el_time]})

        self.df = pd.concat([self.df, line])
        self.df.to_csv(self.filepath, index=False)

        return significance



for i in range(4):    
    if i == 0:
        #sampler = optuna.samplers.NSGAIISampler(seed=11)
        sampler = optuna.samplers.TPESampler(consider_prior = True, seed=11 )
        study = optuna.create_study(direction='maximize', sampler = sampler)
    else:
        study = joblib.load(study_filename)
    
    study_filename = "tpe_sampler_sig/study" + str(i)
    df_filename = "tpe_sampler_sig/df" + str(i)
    
    
    study.optimize(CObjective(df_filename), n_trials=500)    

    joblib.dump(study, study_filename)

    
print(optuna.importance.get_param_importances(study))
print(optuna.importance.MeanDecreaseImpurityImportanceEvaluator().evaluate(study))
print(optuna.importance.FanovaImportanceEvaluator().evaluate(study))

study.trials_dataframe().to_csv('tpe_sampler_sig/trials_dataframe.csv')